### AB Model from spec

Package installation

In [1]:
# !pip3 install seaborn
# !pip3 install python-statemachine
# !pip3 install mesa
# !pip3 install transitions
# !pip3 install scipy
# !pip3 install cufflinks
# !pip3 install graphviz pygraphviz
# !pip3 install graphviz
# !pip3 install transitions[diagrams]
# !pip3 uninstall mesa

Package importation

In [2]:
# imports
import os
import seaborn as sns
from random import choice
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import mesa
from mesa import Agent, Model
from mesa.time import RandomActivation, RandomActivationByType, SimultaneousActivation
from mesa.datacollection import DataCollector
from matplotlib import pyplot as plt, patches
import scipy.stats as ss
import cufflinks as cf
cf.go_offline()
from plotly.offline import iplot
from transitions import Machine
import random
from transitions.extensions import GraphMachine
import graphviz
import timeit
import logging
os.environ["PATH"] += os.pathsep + '/Users/isaacao/homebrew/bin' # for graphviz

Model component importation

In [3]:
from EV.agent import EV, Cpoint
import EV.model as model
from EV.statemachine import EVSM
from EV.model import get_evs_charge, get_evs_charge_level, get_evs_active, get_evs_charging, get_evs_queue, get_evs_travel, get_evs_not_idle, get_active_cpoints, get_eod_evs_socs, get_evs_destinations, get_ev_distance_covered

State diagram

### Model

Model parameters

In [4]:
ticks =  23 # 3600 ticks = 3600 seconds = 1 hour
no_evs = 3
# no of cps must be fixed
no_cps = 5

Run Model

In [5]:
model_run = model.EVModel(ticks=ticks, no_evs=no_evs, no_cps=no_cps)
for i in range(ticks):
    model_run.step()

EV No 1 initialized. Journey type: Urban. Vehicle State: Idle
EV info: ID: 0, destination name: market , distance goal: 80, max_battery: 56, speed: 10, energy consumption rate 0.2
EV No 2 initialized. Journey type: InterUrban. Vehicle State: Idle
EV info: ID: 1, destination name: City A , distance goal: 210, max_battery: 62, speed: 20, energy consumption rate 0.5
EV No 3 initialized. Journey type: InterUrban. Vehicle State: Idle
EV info: ID: 2, destination name: City C , distance goal: 245, max_battery: 50, speed: 20, energy consumption rate 0.5
CP info: ID: 3, initialized. Charge rate: 7.5 kW.
CP info: ID: 4, initialized. Charge rate: 7.5 kW.
CP info: ID: 5, initialized. Charge rate: 7.5 kW.
CP info: ID: 6, initialized. Charge rate: 7.5 kW.
CP info: ID: 7, initialized. Charge rate: 7.5 kW.
CP: 3 is at checkpoint: 40 miles
CP: 4 is at checkpoint: 80 miles
CP: 5 is at checkpoint: 120 miles
CP: 6 is at checkpoint: 160 miles
CP: 7 is at checkpoint: 200 miles
Model initialised. 3 EVs and 5

MachineError: "Can't trigger event end_travel from state Travel_low!"

In [ ]:
run_stats = model_run.datacollector.get_model_vars_dataframe()
print(run_stats)

In [ ]:
a = 26 % 24
print(a) 

Export results to CSV file

In [ ]:
model_run.datacollector.get_model_vars_dataframe().to_csv('model_output.csv')

### Batching

In [ ]:
from mesa.batchrunner import BatchRunner

EVcounts = (100,500,1000)
cpcounts = (1,2)
tickcounts = (24,48)
model_reporters={'EVs Charged': get_evs_charged,
                'EVs Activated': get_evs_active,
                'EVs Travelling': get_evs_travel,
                'EVs Charge Level': get_evs_charge_level,
                'EVs Currently charging': get_evs_charging,
                'EVs Not Idle': get_evs_not_idle,
                'EOD Battery SOC': get_eod_evs_socs,
                'EVs Destinations': get_evs_destinations,
                }
# parameters = {"no_evs": range(1000,20000,3000), "no_cps": 1}
parameters = {"no_evs": EVcounts, "no_cps": cpcounts, "ticks": tickcounts}
batch_run = BatchRunner(model.EVModel, parameters, max_steps=24, iterations=1, model_reporters= model_reporters) #iterations=1
batch_run.run_all()

In [ ]:
# batch_df = batch_run.get_model_vars_dataframe()

In [ ]:
# print(batch_df)

### Visualisations

### Scrapbook